## Tassi di Mortalità

In [1]:
# Import librerie per analisi dati (Pandas) e dati Istat
import os
import pandas as pd
import numpy as np
from IPython.core.display import HTML
import istat
import jsonstat

# cache dir per velocizzare analisi in locale
cache_dir = os.path.abspath(os.path.join("..", "tmp/od_la_grande_fuga", "istat_cached"))
istat.cache_dir(cache_dir)
istat.lang(0)  # lingua italiano
# print("cache_dir is '{}'".format(istat.cache_dir()))

cache_dir is '/home/acesarini/opendata/tmp/od_la_grande_fuga/istat_cached'


In [17]:
dir_df = os.path.join(os.path.abspath(''),'stg')

In [2]:
istat_area_pop = istat.area(8)

In [3]:
istat_dataset_popolazione = istat_area_pop.dataset('DCIS_MORTALITA1')
istat_dataset_popolazione

nr,name,nr. values,values (first 3 values)
0,Territorio,140,"1:'Italia', 3:'Nord', 4:'Nord-ovest' ..."
1,Funzioni biometriche,6,"58:'sopravviventi - lx', 59:'decessi - dx', 60:'probabilità di morte (per 1.000) - qx' ..."
2,Sesso,2,"1:'maschi', 2:'femmine'"
3,Età e classi di età,144,"1:'0 anni', 2:'1 anni', 3:'2 anni' ..."
4,Anno,42,"1920:'2001', 1730:'1991', 1540:'1981' ..."


In [72]:
# istat_dataset_popolazione.dimensions()

In [57]:
# Numero di Morti per Classi di Età nel 2015 Italia
spec_it = {
    "Territorio":1,                 
    "Funzioni biometriche":59,            
    #"Sesso":1, 
    #"Età e classi di età":1
    "Anno": 2186
}

In [58]:
# Numero di Morti per Classi di Età nel 2015 Nord
spec_n = {
    "Territorio":3,                 
    "Funzioni biometriche":59,            
    #"Sesso":1, 
    #"Età e classi di età":1
    "Anno": 2186
}

In [59]:
# Numero di Morti per Classi di Età nel 2015 Sud
spec_s = {
    "Territorio":89,                 
    "Funzioni biometriche":59,            
    #"Sesso":1, 
    #"Età e classi di età":1
    "Anno": 2186
}

In [64]:
def get_df(spec, value):
    c = istat_dataset_popolazione.getvalues(spec)
    ds = c.dataset(0)
    df = ds.to_data_frame('Sesso')
    df.columns = ['Eta', value]
    df=df[(df['Eta']=='fino a 4 anni') |
       (df['Eta']=='5-9 anni') |
       (df['Eta']=='10-14 anni') |
       (df['Eta']=='15-19 anni') |
       (df['Eta']=='20-24 anni') |
       (df['Eta']=='25-29 anni') |
       (df['Eta']=='30-34 anni') |
       (df['Eta']=='35-39 anni') |
       (df['Eta']=='40-44 anni') |
       (df['Eta']=='45-49 anni') |
       (df['Eta']=='50-54 anni') |
       (df['Eta']=='55-59 anni') |
       (df['Eta']=='60-64 anni') |
       (df['Eta']=='65-69 anni') |
       (df['Eta']=='70-74 anni') |
       (df['Eta']=='75-79 anni') |
       (df['Eta']=='80-84 anni') |
       (df['Eta']=='85-89 anni') 
      ]
    df = df.groupby('Eta',as_index=False)
    df = df.agg({value : np.sum})
    return df

In [88]:
df_it = get_df(spec_it, 'Value_IT')
df_n = get_df(spec_n, 'Value_N')
df_s = get_df(spec_s, 'Value_S')

In [89]:
df = pd.merge(df_it,df_n, how='inner', on=['Eta', 'Eta'])

In [90]:
df = pd.merge(df,df_s, how='inner', on=['Eta', 'Eta'])

In [91]:
df.Eta.replace(['fino a 4 anni','5-9 anni'], ['00-04 anni','05-09 anni'], inplace=True)

In [94]:
df = df.sort_values(by=['Eta'], ascending=[1])

In [95]:
df_filename = r'df_morte.pkl'
df_fullpath = os.path.join(dir_df, df_filename)
df.to_pickle(df_fullpath)

In [18]:
# writer = pd.ExcelWriter('Eta.xlsx')
# df.to_excel(writer,'eta')
# writer.save()